<a href="https://colab.research.google.com/github/Srivatsav515/NLP_3/blob/main/Assignment_8_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for i in range(1, len(tokenizer.texts_to_sequences([text])[0])):
    n_gram_sequence = tokenizer.texts_to_sequences([text])[0][:i+1]
    input_sequences.append(n_gram_sequence)

max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

def transformer_block(vocab_size, seq_len):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(vocab_size, 64)(inputs)

    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=64)(embedding, embedding)
    attention_output = tf.keras.layers.LayerNormalization()(attention_output + embedding)

    ff_output = tf.keras.layers.Dense(128, activation='relu')(attention_output)
    ff_output = tf.keras.layers.Dense(64)(ff_output)
    ff_output = tf.keras.layers.LayerNormalization()(ff_output + attention_output)

    outputs = tf.keras.layers.GlobalAveragePooling1D()(ff_output)
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(outputs)

    model = tf.keras.models.Model(inputs, outputs)
    return model

model = transformer_block(total_words, max_seq_len-1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

epochs_list = [20, 60, 70]
history = []
for epoch in epochs_list:
    print(f"Training with {epoch} epochs...")
    h = model.fit(X, y, epochs=epoch, verbose=1)
    history.append(h)

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)
        output_word = tokenizer.index_word[predicted_word_index[0]]
        seed_text += " " + output_word
    return seed_text

seed_text = "Once upon a time"
generated_text = generate_text(seed_text, 50, model, max_seq_len)
print("Generated text:", generated_text)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 55)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 55, 64)               2624      ['input_1[0][0]']             
                                                                                                  
 multi_head_attention (Mult  (None, 55, 64)               33216     ['embedding[0][0]',           
 iHeadAttention)                                                     'embedding[0][0]']           
                                                                                                  
 tf.__operators__.add (TFOp  (None, 55, 64)               0         ['multi_head_attention[0][